In [4]:
!pip install -q langchain faiss-cpu unstructured PyPDF2
!pip install -q huggingface_hub
!pip install -U langchain-community langchain-huggingface
!pip install -q langchain-huggingface
!pip install transformers datasets tqdm ddgs
!pip install langchain-embedding
!pip install InstructorEmbedding

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 7.8 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.11 which is incompatible.
langchain-huggingface 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-community 0.4.1 requires langchain-core<2.0.0,>=1.0.1, but you have langchain-core 0.3.79 which is incompatible.
  Using cached langchain_core-1.0.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached langchain_text_splitters-1.0.0-py3-none-any.whl.metadata (2.6 kB)
Using cached langchain_core-1.0.1-py3-none-any.whl (467 kB)
Using cached l

In [2]:
import os
import json
from tqdm import tqdm
from langchain.vectorstores import FAISS

2025-10-27 21:56:37.174497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761602197.393423      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761602197.454811      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Model Load

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# === 1️⃣ Load fine-tuned financial LLaMA3 model for .invoke() ===
llm_model_path = "/kaggle/input/investing-fine-tuned-model-llama3-2/other/default/1"
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_path)
llm_model = AutoModelForCausalLM.from_pretrained(llm_model_path, torch_dtype=torch.float16, device_map="auto")

llm_pipeline = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=llm_tokenizer,
    device_map="auto",
    torch_dtype=torch.float16,
    max_new_tokens=256,
    temperature=0.2,
    do_sample=False
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)
print("✅ Fine-tuned financial model loaded.")

# === 2️⃣ Load local INSTRUCTOR embedding model ===
embedding_model_path = "/kaggle/input/qwen-3-embedding/transformers/4b/1"
embeddings_model = INSTRUCTOR(embedding_model_path, device=device)
print("✅ Embedding model loaded successfully.")

# LLM
response = llm.invoke("Get Tesla 2023 annual report")
print(response)

# Embeddings
vector = embeddings_model.encode("Tesla revenue 2023")
print(len(vector))

Using device: cpu


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipykernel_37/2433458544.py:24: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


✅ Fine-tuned financial model loaded.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

`generation_config` default values have been modified to match model-specific defaults: {'do_sample': True}. If this is not desired, please set these values explicitly.


✅ Embedding model loaded successfully.
Get Tesla 2023 annual report
The Tesla 2023 annual report is available. It covers the company's financial performance, product offerings, and future plans. It also includes information about the company's commitment to sustainability and its efforts to reduce environmental impact. The report is available in PDF format. (Note: The exact date of the report's release is not provided in the given text.) The report is also available in digital format, but the exact date of its release is not provided in the given text. (Note: The exact date of the report's release is not provided in the given text.) The report is also available in digital format, but the exact date of its release is not provided in the given text.) The report is also available in digital format, but the exact date of its release is not provided in the given text.) The report is also available in digital format, but the exact date of its release is not provided in the given text.) The r

# Common Functions

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from typing import List

def get_split_docs(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=350
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Created {len(split_docs)} text chunks.")
    return split_docs


# Example documents
docs = [
    Document(page_content="This is the content of document 1."),
    Document(page_content="This is the content of document 2.")
]

chunks = split_text(docs)
print(chunks[0].page_content)  # first chunk

Created 2 text chunks.
This is the content of document 1.


In [ ]:
def get_combined_vector_db(*args):
    combined_docs = []
    for doc_list in args:
        if doc_list:  # make sure it's not None or empty
            combined_docs.extend(doc_list)
    
    # Create vector DB for all docs
    combined_vector_db = FAISS.from_documents(combined_docs, embeddings)
    
    return combined_vector_db


In [ ]:
def get_pdf_split_docs(pdf_folder):
    pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]
    all_documents = []
    
    for pdf_path in pdf_files:
        loader = PyPDFLoader(pdf_path)
        docs = loader.load()  # this returns list of Document objects
        all_documents.extend(docs)
        
    split_docs = get_split_docs(all_documents)
    
    return split_docs

# Three sources of information will be used. 
### 1. PDF books added to Kaggle input directory
### 2. Web search results
### 3. Company Annual Reports

# 1. PDF Embedding

In [ ]:
pdf_folder = "/kaggle/input/investing-books-pdf"
local_pdf_split_docs = get_pdf_split_docs(pdf_folder)

def get_local_pdf_split_docs(query):
    return local_pdf_split_docs

# 2. Web Search

In [15]:
from ddgs import DDGS
from langchain.schema import Document
import requests
from bs4 import BeautifulSoup

def web_search(query, max_results=3):
    docs = []

    # Search the web using DuckDuckGo
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=max_results))

    for result in results:
        url = result.get("href")
        title = result.get("title")

        try:
            # Fetch the web page content
            response = requests.get(url, timeout=10)
            response.raise_for_status()

            # Parse visible text from HTML
            soup = BeautifulSoup(response.text, "html.parser")
            for tag in soup(["script", "style", "noscript"]):
                tag.extract()
            text = " ".join(soup.get_text(separator=" ").split())

            # Only keep if text length is reasonable
            if len(text) > 500:
                docs.append(Document(page_content=text, metadata={"source": url, "title": title}))
        except Exception as e:
            # print(f"⚠️ Skipping {url}: {e}")

    # print(f"✅ Retrieved {len(docs)} web documents from top {max_results} results.")
    return docs

query = "Tesla 2025 annual revenue analysis"
documents = web_search(query)

print(f"Number of documents: {len(documents)}")
print(documents[0].metadata)
print(documents[0].page_content[:500])  # preview text

✅ Retrieved 3 web documents from top 3 results.
Number of documents: 3
{'source': 'https://stockanalysis.com/stocks/tsla/revenue/', 'title': 'Tesla (TSLA) Revenue 2015-2025'}
Tesla (TSLA) Revenue 2015-2025 Skip to main content Log In Sign Up Home Watchlist Stocks Stock Screener Stock Exchanges Comparison Tool Earnings Calendar By Industry Stock Lists Top Analysts Top Stocks Corporate Actions IPOs Recent IPOs IPO Calendar IPO Statistics IPO News IPO Screener ETFs ETF Screener Comparison Tool New Launches ETF Providers News Trending Articles Technical Chart Market Movers Top Gainers Top Losers Most Active Premarket After Hours Market Heatmap Market Newsletter Stock Ana


In [ ]:
def get_web_search_split_docs(query, max_results=3):
    docs = web_search(query, max_results)
    split_docs = get_split_docs(docs)
    return split_docs

# 3. Web PDF Retriever

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json

def analyze_query_with_llm(query: str):
    """
    Use the fine-tuned LLM to decide if annual report retrieval is required,
    and extract the company name and year if applicable.

    Returns a dictionary like:
    {
        "required": True,
        "company": "Tesla",
        "year": "2024"
    }
    """
    prompt = f"""
    You are an intelligent financial assistant. 
    Your task is to analyze the user query and decide:
    1. Whether it requires downloading a company's annual report.
    2. If yes, extract the company name and the year of the report.
    
    Respond strictly in this JSON format:
    {{
      "required": true or false,
      "company": "Company name or null",
      "year": "Year or null"
    }}
    
    User query: "{query}"
    """
    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate model output
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.2,
        do_sample=False
    )

    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Try to extract JSON response from model output
    try:
        json_start = response_text.find("{")
        json_end = response_text.rfind("}") + 1
        json_str = response_text[json_start:json_end]
        result = json.loads(json_str)
    except Exception as e:
        print("⚠️ Could not parse model output properly. Raw output:")
        print(response_text)
        result = {"required": False, "company": None, "year": None}

    return result


In [ ]:
from ddgs import DDGS

def search_pdfs(company_name, year=2024, max_results=5):
    query = f"{company_name} {year} annual report filetype:pdf"
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=max_results))
    pdf_links = [r['href'] for r in results if r['href'].endswith('.pdf')]
    return pdf_links

# Example:
pdf_urls = search_pdfs("Square Pharma", 2024)
print(pdf_urls)

In [ ]:
import requests

def download_pdf(pdf_links, save_path):
    for url in pdf_links:    
        response = requests.get(url, stream=True)
        if response.status_code == 200 and 'application/pdf' in response.headers.get('Content-Type', ''):
            with open(save_path, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            print(f"✅ Downloaded: {save_path}")
        else:
            print("❌ Not a valid PDF or download failed.")
# if pdf_urls:
    # for i in range(len(pdf_urls)):
        # download_pdf(pdf_urls[4], "apple_2024_annual_report.pdf")
# download_pdf("https://www.squarepharma.com.bd/downloads/Square%20Pharma_AR_24%20dt-24-11-24_compressed_1.pdf", "hjd.pdf")

In [ ]:
def get_web_pdf_split_docs(query, max_results=5):
    result = analyze_query_with_llm(query) 
    
    pdf_links = search_pdfs(result['company'], result['year'], max_results)
    
    output_dir = "/kaggle/working/"
    
    download_pdf(pdf_links)
    
    split_docs = get_pdf_split_docs(output_dir)

    return split_docs
    

# Retrieval

In [ ]:
def rag_retrieval(vector_db, query):
    """
    Perform similarity search on the vector DB using the query.
    Returns top relevant documents.
    """
    return vector_db.similarity_search(query, k=5)  # top 5 results


# Augmentation

In [ ]:
from langchain.prompts import PromptTemplate

# Define the template string
prompt = """
You are a financial and investing expert specializing in long-term investing, company analysis, and stock market strategies. 
You have deep knowledge of business fundamentals, financial statements, market trends, and investment analysis.

Use the following context from company reports, web data, and relevant documents to answer the user query. 
Always base your answers on the provided context and do not make unsupported claims. 

Context:
{context}

User Question:
{question}

Instructions:
- Analyze the financial and business information carefully.
- Provide long-term investment insights.
- Give clear reasoning and avoid generic statements.
- If the context does not provide sufficient information, say "Insufficient data provided."
- Summarize your analysis in a professional and concise manner.

Answer:
"""

# Create a PromptTemplate instance
investing_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt
)

# Example usage
retrieved_context = "Tesla's revenue in 2023 increased by 15% compared to 2022. Gross margin is 25%."
user_query = "Should I invest in Tesla for the next 5 years?"

final_prompt = investing_prompt.format(context=retrieved_context, question=user_query)
print(final_prompt)

# Generation

In [ ]:
query = "Give me detailed fundamental analysis on Marico Bangladesh Limited"

In [ ]:
context = rag_retrieval(vector_db, query)
final_prompt = investing_prompt.format(context=retrieved_context, question=user_query)

In [ ]:
ans = llm.invoke(final_prompt)
print(ans)

# Chain Construction

In [ ]:
# from langchain.schema import Document
# from langchain.runnables import (
#     RunnablePassthrough,
#     RunnableLambda,
#     RunnableSequence,
#     RunnableParallel,
# )

# # 2️⃣ Parallel execution of three branches
# parallel_chain = RunnableParallel(
#     {
#         "books": RunnableSequence(
#             [
#                 RunnableLambda(lambda query: get_local_pdf_split_docs(query)),   # split preloaded book PDFs
#             ]
#         ),
#         "online_text": RunnableSequence(
#             [
#                 RunnableLambda(lambda query: get_web_search_split_docs(query)),  # search + split text
#             ]
#         ),
#         "online_pdf": RunnableSequence(
#             [
#                 RunnableLambda(lambda query: get_web_pdf_split_docs(query)),  # download + split PDFs
#             ]
#         ),
#     },
#     combine_mode="list",  # returns a list of outputs for each branch
# )

# # 3️⃣ Combine embeddings and build RAG pipeline
# combine_chain = RunnableSequence(
#     [
#         RunnableLambda(lambda split_docs_lists: get_combined_vector_db(*split_docs_lists)),  # combine all split docs
#         RunnableLambda(lambda vector_db: rag_retrieval(vector_db)),                           # retrieval + augmentation
#         RunnableLambda(lambda result: generate_answer(result)),                               # LLM generation
#     ]
# )

# # 4️⃣ Full pipeline: sequence of parallel + combine
# full_pipeline = RunnableSequence(
#     [
#         RunnablePassthrough(),

#         # Step 2: Parallel execution
#         RunnableLambda(
#             lambda query: parallel_chain.invoke({
#                 "books": query,  # load local book PDFs internally
#                 "online_text": query,                # user query for web search
#                 "online_pdf": query                  # user query for online PDFs
#             })
#         ),

#         # Step 3: Combine embeddings and run RAG
#         combine_chain
#     ]
# )